In [305]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [145]:
#getting train and test data
df = pd.read_csv("train.csv")
tst = pd.read_csv('test.csv')

df.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [146]:
#chcecking for missed values
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [147]:
#creating a new feature called deck using cabin feature
import re         #regular expression librabry to do .map()
deck = {"A":1,"B":2,"C":3,"D":4,"E":5,"F":6,"G":7,"U":8}
data = [df,tst]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    #creating the new column wherever there is a letter in it
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    #mapping according to deck
    dataset['Deck'] = dataset['Deck'].map(deck)
    
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
    
 

In [148]:
#dropping the cabin column 
df = df.drop(["Cabin"],axis = 1)
tst = tst.drop(["Cabin"],axis = 1)

In [149]:
#filling the age feature's missing value
data = [df,tst]
for i in data:
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = i["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values
    age_slice = i["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    i["Age"] = age_slice
    i["Age"] = i["Age"].astype(int)
df["Age"].isnull().sum()  #no null values in Age column


0

In [150]:
#About embarked column 
df["Embarked"].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [151]:
#filling the null values with most occuring
data = [df,tst]

for i in data:
    i["Embarked"] = i["Embarked"].fillna("S")
    #changeing fare data type to int
    i["Fare"] = i["Fare"].fillna(0)
    i["Fare"] = i["Fare"].astype(int)
    
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  891 non-null    int64 
 1   Survived     891 non-null    int64 
 2   Pclass       891 non-null    int64 
 3   Name         891 non-null    object
 4   Sex          891 non-null    object
 5   Age          891 non-null    int32 
 6   SibSp        891 non-null    int64 
 7   Parch        891 non-null    int64 
 8   Ticket       891 non-null    object
 9   Fare         891 non-null    int32 
 10  Embarked     891 non-null    object
 11  Deck         891 non-null    int32 
dtypes: int32(3), int64(5), object(4)
memory usage: 73.2+ KB


In [152]:
#changing name column into numaric value
data = [df,tst]
title = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Others": 5}


for i in data:
        # extract titles
        i['Title'] = i.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
        # replace titles with a more common title or as others
        i['Title'] = i['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Others')
        
        
        i['Title'] = i['Title'].replace('Mlle', 'Miss')
        i['Title'] = i['Title'].replace('Ms', 'Miss')
        i['Title'] = i['Title'].replace('Mme', 'Mrs')
        # convert titles into numbers
        i['Title'] = i['Title'].map(title)
        # filling NaN with 0
        i['Title'] = i['Title'].fillna(0)
        
#dropping name feature
df = df.drop(['Name'], axis=1)
tst = tst.drop(['Name'], axis=1)
    
 

In [153]:
#sex and embarked into numbers 
data = [df,tst]
s = {"male" : 0,"female" : 1}
e = {"S": 0, "C": 1, "Q": 2}

for i in data:
    i["Sex"] = i["Sex"].map(s)
    i["Embarked"] = i["Embarked"].map(e)
    
df["Sex"]

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    1
889    0
890    0
Name: Sex, Length: 891, dtype: int64

In [154]:
#dropping ticket

df = df.drop(["Ticket"],axis =1)
tst = tst.drop(["Ticket"],axis = 1)



In [163]:
print(df["Age"].describe())
lbl = pd.qcut(df['Age'], q=8, labels=['a', 'b', 'c','d','e','f','g','h'])
print(lbl.value_counts())

count    891.000000
mean      29.468013
std       13.497142
min        0.000000
25%       21.000000
50%       28.000000
75%       37.000000
max       80.000000
Name: Age, dtype: float64
a    129
b    116
d    115
f    115
g    114
e    108
h    101
c     93
Name: Age, dtype: int64


In [167]:
#grouping age catagory

data = [df, tst]
for i in data:
    i['Age'] = i['Age'].astype(int)
    i.loc[ i['Age'] <= 11, 'Age'] = 0  #df.loc[row_labels, column_labels]     The .loc function is a way to subset rows and columns of a Pandas DataFrame by label-based indexin
    i.loc[(i['Age'] > 11) & (i['Age'] <= 18), 'Age'] = 1   # sets the values of the 'Age' column to 1 for all rows where the 'Age' value is greater than 11 and less than or equal to 18.
    i.loc[(i['Age'] > 18) & (i['Age'] <= 22), 'Age'] = 2
    i.loc[(i['Age'] > 22) & (i['Age'] <= 27), 'Age'] = 3
    i.loc[(i['Age'] > 27) & (i['Age'] <= 33), 'Age'] = 4
    i.loc[(i['Age'] > 33) & (i['Age'] <= 40), 'Age'] = 5
    i.loc[(i['Age'] > 40) & (i['Age'] <= 66), 'Age'] = 6
    i.loc[ i['Age'] > 66, 'Age'] = 7
    
print( df["Age"].value_counts())


df.info()

4    162
6    155
5    145
3    143
2    117
1     94
0     68
7      7
Name: Age, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  891 non-null    int64
 1   Survived     891 non-null    int64
 2   Pclass       891 non-null    int64
 3   Sex          891 non-null    int64
 4   Age          891 non-null    int32
 5   SibSp        891 non-null    int64
 6   Parch        891 non-null    int64
 7   Fare         891 non-null    int32
 8   Embarked     891 non-null    int64
 9   Deck         891 non-null    int32
 10  Title        891 non-null    int64
dtypes: int32(3), int64(8)
memory usage: 66.3 KB


In [166]:
print(df["Fare"].describe())
lbl = pd.qcut(df['Fare'], q=6, labels=['a', 'b', 'c','d','e','f'])
print(lbl.value_counts())

count    891.000000
mean      31.785634
std       49.703730
min        0.000000
25%        7.000000
50%       14.000000
75%       31.000000
max      512.000000
Name: Fare, dtype: float64
a    241
d    165
c    146
f    146
e    123
b     70
Name: Fare, dtype: int64


In [168]:
#catogorizing fare values

data = [df, tst]

for i in data:
    i.loc[ i['Fare'] <= 8.0, 'Fare'] = 0
    i.loc[(i['Fare'] > 8.0) & (i['Fare'] <= 14.5), 'Fare'] = 1
    i.loc[(i['Fare'] > 14.5) & (i['Fare'] <= 31), 'Fare']   = 2
    i.loc[(i['Fare'] > 31) & (i['Fare'] <= 99), 'Fare']   = 3
    i.loc[(i['Fare'] > 99) & (i['Fare'] <= 250), 'Fare']   = 4
    i.loc[ i['Fare'] > 250, 'Fare'] = 5
    i['Fare'] = i['Fare'].astype(int)

In [179]:
df.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,Title,Age_Class,Age_class,Fare_Per_Person
0,1,0,3,0,2,1,0,0,0,8,1,6,6,0
1,2,1,1,1,5,1,0,3,1,3,3,5,5,1
2,3,1,3,1,3,0,0,0,0,8,2,9,9,0
3,4,1,1,1,5,1,0,3,0,3,3,5,5,1
4,5,0,3,0,5,0,0,0,0,8,1,15,15,0


In [170]:
#creating a new feature   Age*Class
data = [df, tst]
for i in data:
    i['Age_class']= i['Age']* i['Pclass']
    
df["Age_class"]

0       6
1       5
2       9
3       5
4      15
       ..
886     6
887     2
888    15
889     3
890    12
Name: Age_class, Length: 891, dtype: int64

In [174]:
#creating a new feature    Fare/Person

for i in data:
    i['Fare_Per_Person'] = i['Fare']/(i['SibSp']+1)
    i['Fare_Per_Person'] = i['Fare_Per_Person'].astype(int)


In [181]:
# Let's take a last look at the training set, before we start training the models.
print(df.head(10))
print(tst.head(10))

   PassengerId  Survived  Pclass  Sex  Age  SibSp  Parch  Fare  Embarked  \
0            1         0       3    0    2      1      0     0         0   
1            2         1       1    1    5      1      0     3         1   
2            3         1       3    1    3      0      0     0         0   
3            4         1       1    1    5      1      0     3         0   
4            5         0       3    0    5      0      0     0         0   
5            6         0       3    0    3      0      0     0         2   
6            7         0       1    0    6      0      0     3         0   
7            8         0       3    0    0      3      1     2         0   
8            9         1       3    1    3      0      2     1         0   
9           10         1       2    1    1      1      0     2         1   

   Deck  Title  Age_Class  Age_class  Fare_Per_Person  
0     8      1          6          6                0  
1     3      3          5          5               

In [331]:
#train test split
x = df.drop(columns=["Survived","PassengerId","Parch","Age"], axis=1)  #Parch,Age are removed because it has low feature importance
y = df["Survived"]


X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.20, random_state = 25)


In [332]:
#using logistic regression
model = LogisticRegression()
model.fit(X_train,Y_train)

Y_pred = model.predict(X_test)

acc_log = round(model.score(X_train, Y_train) * 100, 3)

In [333]:
#using decision tree

dt = DecisionTreeClassifier() 
dt.fit(X_train, Y_train)  
Y_pred = dt.predict(X_test)  
acc_decision_tree = round(dt.score(X_train, Y_train) * 100, 3)

In [334]:
#using random forest 

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, Y_train)

Y_pred = rf.predict(X_test)

acc_random_forest = round(rf.score(X_train, Y_train) * 100, 3)

print(Y_pred)

[0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0
 0 1 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1
 0 1 0 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0
 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0]


In [335]:
print(f"Accuracy for Logistic Regression model is = {acc_log} %")
print(f"Accuracy for Decision Tree model is = {acc_decision_tree} %")
print(f"Accuracy for Random forest model is = {acc_random_forest} %")

Accuracy for Logistic Regression model is = 81.461 %
Accuracy for Decision Tree model is = 92.837 %
Accuracy for Random forest model is = 92.837 %


In [337]:
#finding feature importance to make the model less over fitting
importances = pd.DataFrame({"feature name":x.columns,"importances":(rf.feature_importances_)})

importances.head(30)



,feature name,importances
0,Pclass,0.074284
1,Sex,0.178767
2,SibSp,0.071147
3,Fare,0.072379
4,Embarked,0.064412
5,Deck,0.081238
6,Title,0.206255
7,Age_Class,0.092909
8,Age_class,0.089159
9,Fare_Per_Person,0.069450
